# 處理 outliers
* 新增欄位註記
* outliers 或 NA 填補
    1. 平均數 (mean)
    2. 中位數 (median, or Q50)
    3. 最大/最小值 (max/min, Q100, Q0)
    4. 分位數 (quantile)

# [作業目標]
- 仿造範例的資料操作, 試著進行指定的離群值處理

# [作業重點]
- 計算 AMT_ANNUITY 的分位點 (q0 - q100) (Hint : np.percentile, In[3])
- 將 AMT_ANNUITY 的 NaN 用中位數取代 (Hint : q50, In[4])
- 將 AMT_ANNUITY 數值轉換到 -1 ~ 1 之間 (Hint : 參考範例, In[5])
- 將 AMT_GOOD_PRICE 的 NaN 用眾數取代 (In[6])

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# 設定 data_path
dir_data = '/Users/kuangtinghsiao/Documents/資料科學馬拉松/Part01/'

In [2]:
f_app = os.path.join(dir_data, 'application_train.csv')
print('Path of read in data: %s' % (f_app))
app_train = pd.read_csv(f_app)
app_train.head()

Path of read in data: /Users/kuangtinghsiao/Documents/資料科學馬拉松/Part01/application_train.csv


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## 1. 列出 AMT_ANNUITY 的 q0 - q100
## 2.1 將 AMT_ANNUITY 中的 NAs 暫時以中位數填補
## 2.2 將 AMT_ANNUITY 的數值標準化至 -1 ~ 1 間
## 3. 將 AMT_GOOD_PRICE 的 NAs 以眾數填補


In [3]:
"""
YOUR CODE HERE
"""
# 1: 計算 AMT_ANNUITY 的 q0 - q100
q_0_to_100=np.arange(101)
q_all =[np.percentile(app_train[~app_train['AMT_ANNUITY'].isnull()]['AMT_ANNUITY'],i ) for i in q_0_to_100]

pd.DataFrame({'q': list(range(101)),
              'value': q_all})

,q,value
0,0,1615.50
1,1,6182.91
2,2,6750.00
3,3,7875.00
4,4,8703.00
5,5,9000.00
6,6,9000.00
7,7,9553.50
8,8,10125.00
9,9,10503.00


In [4]:
# 2.1 將 NAs 以 q50 填補
print("Before replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))


q_50 = np.percentile(app_train[~app_train['AMT_ANNUITY'].isnull()]['AMT_ANNUITY'], 50)
app_train.loc[app_train['AMT_ANNUITY'].isnull(),'AMT_ANNUITY'] = q_50

print("After replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

Before replace NAs, numbers of row that AMT_ANNUITY is NAs: 12
After replace NAs, numbers of row that AMT_ANNUITY is NAs: 0


### Hints: Normalize function (to -1 ~ 1)
$ y = 2*(\frac{x - min(x)}{max(x) - min(x)} - 0.5) $

In [6]:
# 2.2 Normalize values to -1 to 1
print("== Original data range ==")
print(app_train['AMT_ANNUITY'].describe())

def normalize_value(x):
    """
    Your Code Here, compelete this function
    """
    x=(((x-(app_train['AMT_ANNUITY'].min()))/(app_train['AMT_ANNUITY'].max()-app_train['AMT_ANNUITY'].min()))-0.5)*2
    return x

app_train['AMT_ANNUITY_NORMALIZED'] = normalize_value(app_train['AMT_ANNUITY'])

print("== Normalized data range ==")
app_train['AMT_ANNUITY_NORMALIZED'].describe()

== Original data range ==
count    307511.000000
mean      27108.487841
std       14493.461065
min        1615.500000
25%       16524.000000
50%       24903.000000
75%       34596.000000
max      258025.500000
Name: AMT_ANNUITY, dtype: float64
== Normalized data range ==


count    307511.000000
mean         -0.801154
std           0.113049
min          -1.000000
25%          -0.883714
50%          -0.818357
75%          -0.742752
max           1.000000
Name: AMT_ANNUITY_NORMALIZED, dtype: float64

In [9]:
# 3
print("Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train['AMT_GOODS_PRICE'].isnull()))

# 列出重複最多的數值
"""
Your Code Here
"""
from scipy.stats import mode
value_most = mode(app_train['AMT_GOODS_PRICE'])
print(value_most)

mode_goods_price = list(app_train['AMT_GOODS_PRICE'].value_counts().index)
app_train.loc[app_train['AMT_GOODS_PRICE'].isnull(), 'AMT_GOODS_PRICE'] = mode_goods_price[0]

print("After replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train['AMT_GOODS_PRICE'].isnull()))

Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 0
ModeResult(mode=array([450000.]), count=array([26300]))
After replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 0


In [26]:
from collections import defaultdict
mode_count=defaultdict(lambda:0)
for values in app_train['AMT_GOODS_PRICE']:
    mode_count[values] +=1
    
A=sorted(mode_count.items(), key=lambda x:x[1], reverse=True)
A

[(450000.0, 26300),
 (225000.0, 25282),
 (675000.0, 24962),
 (900000.0, 15416),
 (270000.0, 11428),
 (180000.0, 10123),
 (454500.0, 9157),
 (1125000.0, 9050),
 (135000.0, 8206),
 (315000.0, 5225),
 (1350000.0, 5037),
 (679500.0, 4824),
 (238500.0, 4696),
 (360000.0, 4553),
 (247500.0, 4318),
 (463500.0, 3873),
 (540000.0, 3763),
 (495000.0, 3463),
 (202500.0, 3222),
 (90000.0, 3169),
 (405000.0, 2954),
 (472500.0, 2828),
 (157500.0, 2704),
 (585000.0, 2647),
 (337500.0, 2501),
 (229500.0, 2438),
 (112500.0, 2170),
 (630000.0, 2164),
 (1800000.0, 2146),
 (720000.0, 1922),
 (1575000.0, 1875),
 (688500.0, 1824),
 (292500.0, 1708),
 (148500.0, 1674),
 (477000.0, 1611),
 (810000.0, 1568),
 (67500.0, 1505),
 (697500.0, 1408),
 (765000.0, 1396),
 (382500.0, 1396),
 (904500.0, 1375),
 (283500.0, 1372),
 (252000.0, 1368),
 (234000.0, 1279),
 (45000.0, 1169),
 (945000.0, 1161),
 (855000.0, 1113),
 (1129500.0, 1109),
 (913500.0, 1065),
 (328500.0, 1059),
 (459000.0, 1023),
 (193500.0, 1007),
 (10

In [29]:
mode_count

defaultdict(<function __main__.<lambda>()>,
            {351000.0: 345,
             1129500.0: 1109,
             135000.0: 8206,
             297000.0: 544,
             513000.0: 204,
             454500.0: 9157,
             1395000.0: 267,
             1530000.0: 152,
             913500.0: 1065,
             405000.0: 2954,
             652500.0: 475,
             67500.0: 1505,
             697500.0: 1408,
             679500.0: 4824,
             247500.0: 4318,
             387000.0: 464,
             270000.0: 11428,
             157500.0: 2704,
             427500.0: 673,
             927000.0: 594,
             450000.0: 26300,
             225000.0: 25282,
             702000.0: 828,
             675000.0: 24962,
             180000.0: 10123,
             477000.0: 1611,
             360000.0: 4553,
             1125000.0: 9050,
             553500.0: 643,
             540000.0: 3763,
             855000.0: 1113,
             238500.0: 4696,
             598500.0: 408,
   